# Building AI Applications with OpenAI's Responses API

## Introduction

Welcome to this comprehensive guide on using OpenAI's **Responses API**, the modern replacement for the deprecated Assistants API. In this tutorial, we'll explore how to build powerful AI applications that can search through documents and perform complex data analysis.

### What's Changed?

The Assistants API is being deprecated (sunset planned for mid-2026) in favor of the new **Responses API**, which offers:

- **Simpler architecture**: No more complex thread management - just pass `previous_response_id` to continue conversations
- **Unified experience**: Combines the best of Chat Completions and Assistants APIs
- **Stateless design**: Instructions are stateless and apply only to the current request
- **Better performance**: More efficient conversation management
- **Same powerful tools**: File Search and Code Interpreter are both fully supported

### What You'll Learn

By the end of this tutorial, you'll be able to:

1. Use the **File Search** tool to build knowledge-based assistants
2. Leverage the **Code Interpreter** tool for data analysis and visualization
3. Manage vector stores for document retrieval
4. Chain responses to maintain conversation context
5. Combine multiple tools for powerful AI applications

Let's get started!

## Setup

First, let's set up our environment with the required imports and initialize our OpenAI client:

In [ ]:
import os
import getpass

def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")

_set_env("OPENAI_API_KEY")

In [ ]:
from openai import OpenAI
import time

# Initialize the OpenAI client
client = OpenAI()

print("OpenAI client initialized successfully!")

---

## Part 1: File Search with the Responses API

File Search allows your AI to access knowledge from documents you provide. OpenAI automatically handles:
- Parsing and chunking documents
- Creating and storing embeddings
- Performing vector and keyword search
- Retrieving relevant content to answer queries

### Understanding Vector Stores

**Vector Stores** are specialized databases for efficient storage and retrieval of information:
- Files are automatically processed (parsed, chunked, embedded)
- Support both keyword and semantic search
- Can store up to 10,000 files each
- Maximum file size: 512 MB per file

### Creating a Vector Store

Let's create a vector store for our documentation:

In [ ]:
def create_vector_store(store_name: str):
    """Creates a vector store for document storage."""
    vector_store = client.vector_stores.create(name=store_name)
    
    details = {
        "id": vector_store.id,
        "name": vector_store.name,
        "created_at": vector_store.created_at,
        "file_count": vector_store.file_counts.completed
    }
    
    print(f"Created vector store: {vector_store.name}")
    print(f"Vector Store ID: {vector_store.id}")
    
    return details

# Create our first vector store
vector_store_details = create_vector_store("Product Documentation")

### Uploading Files to the Vector Store

Now let's upload documents to our vector store. We'll download a sample research paper from arXiv:

In [ ]:
import requests

# Download a sample research paper from arXiv
paper_url = "https://arxiv.org/pdf/2503.03718"
file_path = "arxiv_paper_sample.pdf"

print("Downloading research paper...")
response = requests.get(paper_url)
response.raise_for_status()

with open(file_path, "wb") as f:
    f.write(response.content)

print(f"Downloaded: {file_path}")

In [ ]:
def upload_file_to_vector_store(file_path: str, vector_store_id: str):
    """Uploads a file to the vector store."""
    file_name = os.path.basename(file_path)
    
    try:
        # Upload file to OpenAI
        print(f"Uploading {file_name}...")
        file_response = client.files.create(
            file=open(file_path, 'rb'),
            purpose="assistants"
        )
        
        # Add file to vector store
        print(f"Adding to vector store...")
        client.vector_stores.files.create(
            vector_store_id=vector_store_id,
            file_id=file_response.id
        )
        
        print(f"✓ Successfully uploaded: {file_name}")
        return {"file": file_name, "status": "success", "file_id": file_response.id}
        
    except Exception as e:
        print(f"✗ Failed to upload {file_name}: {str(e)}")
        return {"file": file_name, "status": "failed", "error": str(e)}

# Upload our file
upload_result = upload_file_to_vector_store(file_path, vector_store_details['id'])

# Wait a moment for processing
print("\nWaiting for file processing...")
time.sleep(3)
print("Ready!")

### Using File Search with the Responses API

Now comes the exciting part! With the Responses API, we can query our documents directly without managing threads or assistants.

The key difference from the old Assistants API:
- **Old way**: Create assistant → Create thread → Add message → Run assistant → Poll for completion
- **New way**: Simply call `responses.create()` with the file_search tool

In [ ]:
def query_documents(query: str, vector_store_id: str, model: str = "gpt-4o"):
    """Query documents using file search in the Responses API."""
    
    instructions = """You are a helpful research assistant. 
    Use the provided documentation to answer questions accurately.
    If you're not sure about something, admit it and stick to the information in the documents.
    Always cite your sources when possible."""
    
    response = client.responses.create(
        input=query,
        model=model,
        instructions=instructions,
        tools=[{
            "type": "file_search",
            "vector_store_ids": [vector_store_id],
            "max_num_results": 5
        }]
    )
    
    return response

# Ask a question about our document
query = "What is this paper about? Provide a brief summary."
print(f"Query: {query}\n")

response = query_documents(query, vector_store_details['id'])

# Display the response
print("Assistant's Response:")
print("="*80)
for item in response.output:
    if hasattr(item, 'content'):
        for content in item.content:
            if hasattr(content, 'text'):
                print(content.text)
print("="*80)

### Continuing the Conversation

One of the most powerful features of the Responses API is simple conversation continuity. Just pass the `previous_response_id` to maintain context:

In [ ]:
def continue_conversation(query: str, previous_response_id: str, vector_store_id: str, model: str = "gpt-4o"):
    """Continue a conversation using previous_response_id."""
    
    instructions = """You are a helpful research assistant. 
    Use the provided documentation to answer questions accurately.
    Refer to our previous conversation when relevant."""
    
    response = client.responses.create(
        input=query,
        model=model,
        instructions=instructions,
        previous_response_id=previous_response_id,
        tools=[{
            "type": "file_search",
            "vector_store_ids": [vector_store_id],
            "max_num_results": 5
        }]
    )
    
    return response

# Ask a follow-up question
follow_up_query = "Can you explain the methodology in more detail?"
print(f"Follow-up Query: {follow_up_query}\n")

follow_up_response = continue_conversation(
    follow_up_query,
    response.id,
    vector_store_details['id']
)

# Display the response
print("Assistant's Response:")
print("="*80)
for item in follow_up_response.output:
    if hasattr(item, 'content'):
        for content in item.content:
            if hasattr(content, 'text'):
                print(content.text)
print("="*80)

### Extracting Citations

The Responses API provides annotations that include information about which files were used to generate the response:

In [ ]:
def extract_citations(response):
    """Extract file citations from response annotations."""
    citations = set()
    
    for item in response.output:
        if hasattr(item, 'content'):
            for content in item.content:
                if hasattr(content, 'annotations'):
                    for annotation in content.annotations:
                        if hasattr(annotation, 'filename'):
                            citations.add(annotation.filename)
    
    return citations

# Extract and display citations
citations = extract_citations(follow_up_response)

if citations:
    print("\nFiles Referenced:")
    for citation in citations:
        print(f"  - {citation}")
else:
    print("\nNo explicit file citations found in response.")

---

## Part 2: Code Interpreter with the Responses API

The Code Interpreter tool enables your AI to write and execute Python code in a secure, sandboxed environment. This is perfect for:
- Data analysis and statistics
- Creating visualizations and charts
- Processing various file formats (CSV, JSON, Excel, etc.)
- Solving complex mathematical problems
- Generating files with data and results

**Cost**: $0.03 per session

### Basic Code Interpreter Usage

Let's start with a simple example:

In [ ]:
def run_code_interpreter(query: str, model: str = "gpt-4o"):
    """Use code interpreter to solve problems."""
    
    instructions = """You are a skilled data analyst and programmer.
    When solving problems:
    1. Write clear, well-documented code
    2. Explain your approach
    3. Show your work step by step
    4. Provide insights from the results"""
    
    response = client.responses.create(
        input=query,
        model=model,
        instructions=instructions,
        tools=[{
            "type": "code_interpreter",
            "container": {"type": "auto"}
        }]
    )
    
    return response

# Example: Solve a math problem
math_query = """Calculate the first 10 Fibonacci numbers and plot them on a graph. 
Also calculate the ratio between consecutive numbers and show how it approaches the golden ratio."""

print(f"Query: {math_query}\n")

math_response = run_code_interpreter(math_query)

# Display the response
print("Assistant's Response:")
print("="*80)
for item in math_response.output:
    if hasattr(item, 'content'):
        for content in item.content:
            if hasattr(content, 'text'):
                print(content.text)
print("="*80)

### Data Analysis with File Upload

Let's create a sample dataset and analyze it using the Code Interpreter:

In [ ]:
import pandas as pd
import numpy as np

# Create a sample dataset
np.random.seed(42)
dates = pd.date_range('2024-01-01', periods=100, freq='D')
data = {
    'date': dates,
    'sales': np.random.randint(100, 1000, 100) + np.arange(100) * 5,
    'expenses': np.random.randint(50, 500, 100) + np.arange(100) * 2,
    'customers': np.random.randint(10, 100, 100)
}

df = pd.DataFrame(data)
csv_file = "sample_business_data.csv"
df.to_csv(csv_file, index=False)

print(f"Created sample dataset with {len(df)} rows")
print(f"\nFirst few rows:")
print(df.head())

In [ ]:
def analyze_data_file(file_path: str, query: str, model: str = "gpt-4o"):
    """Upload a file and analyze it using code interpreter."""
    
    # Upload the file
    file_response = client.files.create(
        file=open(file_path, 'rb'),
        purpose="assistants"
    )
    
    instructions = """You are an expert data analyst.
    Analyze the provided data file and create insightful visualizations.
    Explain your findings clearly with statistical evidence."""
    
    # Create input with file attachment
    input_data = [
        {
            "role": "user",
            "content": [
                {
                    "type": "input_file",
                    "file_id": file_response.id
                },
                {
                    "type": "input_text",
                    "text": query
                }
            ]
        }
    ]
    
    response = client.responses.create(
        input=input_data,
        model=model,
        instructions=instructions,
        tools=[{
            "type": "code_interpreter",
            "container": {"type": "auto"}
        }]
    )
    
    return response, file_response.id

# Analyze our dataset
analysis_query = """Please analyze this business data:
1. Calculate basic statistics (mean, median, trends)
2. Create visualizations showing sales and expenses over time
3. Calculate the profit margin trend
4. Identify any interesting patterns or anomalies
5. Provide actionable insights"""

print(f"Analyzing data...\n")

analysis_response, uploaded_file_id = analyze_data_file(csv_file, analysis_query)

# Display the response
print("Analysis Results:")
print("="*80)
for item in analysis_response.output:
    if hasattr(item, 'content'):
        for content in item.content:
            if hasattr(content, 'text'):
                print(content.text)
print("="*80)

### Handling Generated Files

Code Interpreter can generate files (like images, CSV files, etc.). Here's how to download them:

In [ ]:
def download_generated_files(response, output_dir="output"):
    """Download any files generated by Code Interpreter."""
    
    # Create output directory if it doesn't exist
    os.makedirs(output_dir, exist_ok=True)
    
    downloaded_files = []
    
    for item in response.output:
        if hasattr(item, 'content'):
            for content in item.content:
                # Check for image files
                if hasattr(content, 'image_file'):
                    file_id = content.image_file.file_id
                    file_content = client.files.content(file_id)
                    
                    output_path = os.path.join(output_dir, f"image_{file_id}.png")
                    with open(output_path, 'wb') as f:
                        f.write(file_content.read())
                    
                    downloaded_files.append(output_path)
                    print(f"Downloaded: {output_path}")
                
                # Check for file path annotations (other file types)
                if hasattr(content, 'annotations'):
                    for annotation in content.annotations:
                        if hasattr(annotation, 'file_path'):
                            file_id = annotation.file_path.file_id
                            file_content = client.files.content(file_id)
                            
                            output_path = os.path.join(output_dir, f"file_{file_id}")
                            with open(output_path, 'wb') as f:
                                f.write(file_content.read())
                            
                            downloaded_files.append(output_path)
                            print(f"Downloaded: {output_path}")
    
    return downloaded_files

# Download any generated files from our analysis
print("\nChecking for generated files...")
files = download_generated_files(analysis_response)

if files:
    print(f"\nDownloaded {len(files)} file(s)")
else:
    print("\nNo files were generated")

---

## Part 3: Combining File Search and Code Interpreter

One of the most powerful features is combining both tools. You can search through documents AND perform data analysis in the same request!

In [ ]:
def combined_analysis(query: str, vector_store_id: str, model: str = "gpt-4o"):
    """Use both file_search and code_interpreter together."""
    
    instructions = """You are an advanced research and data analysis assistant.
    Use file_search to find relevant information from documents.
    Use code_interpreter to perform calculations, analysis, and create visualizations.
    Combine insights from both tools to provide comprehensive answers."""
    
    response = client.responses.create(
        input=query,
        model=model,
        instructions=instructions,
        tools=[
            {
                "type": "file_search",
                "vector_store_ids": [vector_store_id],
                "max_num_results": 5
            },
            {
                "type": "code_interpreter",
                "container": {"type": "auto"}
            }
        ]
    )
    
    return response

# Example query that benefits from both tools
combined_query = """Based on the research paper in the knowledge base, 
extract any numerical data or statistics mentioned, and create a visualization 
to illustrate the key findings. Explain what the data shows."""

print(f"Query: {combined_query}\n")

combined_response = combined_analysis(combined_query, vector_store_details['id'])

# Display the response
print("Combined Analysis:")
print("="*80)
for item in combined_response.output:
    if hasattr(item, 'content'):
        for content in item.content:
            if hasattr(content, 'text'):
                print(content.text)
print("="*80)

# Download any generated files
print("\nChecking for generated visualizations...")
download_generated_files(combined_response)

---

## Best Practices and Cost Management

### Cost Considerations

- **File Search**: $2.50 per 1,000 queries + $0.10/GB/day storage (first GB free)
- **Code Interpreter**: $0.03 per session
- **Vector Storage**: Set expiration policies to manage costs

In [ ]:
def create_vector_store_with_expiration(name: str, days: int = 7):
    """Create a vector store with automatic expiration after inactivity."""
    
    vector_store = client.vector_stores.create(
        name=name,
        expires_after={
            "anchor": "last_active_at",
            "days": days
        }
    )
    
    print(f"Created vector store: {name}")
    print(f"Will expire after {days} days of inactivity")
    
    return vector_store

# Example: Create a temporary vector store
temp_store = create_vector_store_with_expiration("Temporary Research Store", days=3)

### Supported File Types

File Search supports many text-based formats:
- Documents: `.pdf`, `.docx`, `.txt`, `.md`
- Code: `.py`, `.js`, `.java`, `.cpp`, `.html`, `.css`
- Data: `.json`, `.csv`, `.xml`
- Maximum file size: 512 MB
- Maximum tokens per file: 5,000,000

### Cleanup Resources

Remember to clean up resources when you're done:

In [ ]:
def cleanup_resources(vector_store_ids=None, file_ids=None):
    """Clean up vector stores and files."""
    
    if vector_store_ids:
        for vs_id in vector_store_ids:
            try:
                client.vector_stores.delete(vs_id)
                print(f"Deleted vector store: {vs_id}")
            except Exception as e:
                print(f"Error deleting vector store {vs_id}: {e}")
    
    if file_ids:
        for file_id in file_ids:
            try:
                client.files.delete(file_id)
                print(f"Deleted file: {file_id}")
            except Exception as e:
                print(f"Error deleting file {file_id}: {e}")

# Example cleanup (uncomment to use)
# cleanup_resources(
#     vector_store_ids=[vector_store_details['id']],
#     file_ids=[uploaded_file_id]
# )

---

## Migration Notes: Assistants API → Responses API

### Key Differences

| Aspect | Assistants API (Old) | Responses API (New) |
|--------|---------------------|--------------------|
| **Conversation Management** | Thread-based, manual history | `previous_response_id` |
| **Instructions** | Stored on server | Stateless, per-request |
| **API Calls** | Create assistant → thread → message → run | Single `responses.create()` |
| **Execution** | Asynchronous with polling | Synchronous or streaming |
| **Complexity** | Higher (multiple objects) | Lower (unified) |

### Code Comparison

**Old Way (Assistants API):**
```python
# Create assistant
assistant = client.beta.assistants.create(
    instructions="You are helpful",
    tools=[{"type": "file_search"}]
)

# Create thread
thread = client.beta.threads.create()

# Add message
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="Hello"
)

# Run assistant
run = client.beta.threads.runs.create_and_poll(
    thread_id=thread.id,
    assistant_id=assistant.id
)

# Get messages
messages = client.beta.threads.messages.list(thread_id=thread.id)
```

**New Way (Responses API):**
```python
# Single call
response = client.responses.create(
    input="Hello",
    instructions="You are helpful",
    tools=[{"type": "file_search", "vector_store_ids": [vs_id]}]
)

# Continue conversation
response2 = client.responses.create(
    input="Follow up",
    previous_response_id=response.id,
    tools=[{"type": "file_search", "vector_store_ids": [vs_id]}]
)
```

### Benefits of Responses API

1. **Simpler code**: Fewer API calls and objects to manage
2. **Better performance**: Direct response without polling
3. **Flexible state**: Choose stateful or stateless as needed
4. **Easier debugging**: Unified request/response structure
5. **Better typing**: Improved TypeScript definitions

---

## Conclusion

Congratulations! You've learned how to use OpenAI's modern Responses API with both File Search and Code Interpreter tools.

### What We Covered

1. **File Search**: Build knowledge-based assistants that can search through documents
2. **Code Interpreter**: Perform data analysis, create visualizations, and solve complex problems
3. **Conversation Management**: Maintain context using `previous_response_id`
4. **Combined Tools**: Use multiple tools together for powerful applications
5. **Best Practices**: Cost management, file handling, and resource cleanup

### Next Steps

- Experiment with your own documents and datasets
- Combine tools creatively for your specific use cases
- Explore the other tools available in the Responses API (web search, etc.)
- Build production applications with proper error handling and monitoring

### Resources

- [OpenAI Responses API Documentation](https://platform.openai.com/docs/api-reference/responses)
- [File Search Guide](https://platform.openai.com/docs/guides/tools-file-search)
- [OpenAI Cookbook Examples](https://cookbook.openai.com/)

Happy building! 🚀